Defines and imports

In [ ]:
import json
from datetime import datetime
from collections import defaultdict
import re
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

PATH = ""

# Load path
with open('.nbconfig', 'r') as file:
    arguments = json.load(file)
    PATH = arguments["path"]

print(f"Report for {PATH}")

with open(PATH + "start.txt") as file:
    simStart = int(file.readline().strip())
with open(PATH + "stop.txt") as file:
    simStop = int(file.readline().strip())


Resource Graphs

In [ ]:
def plotResourceGraphs():
    logfiles = []

    for root, _, files in os.walk(PATH):
        for file in files:
            if file in ["pidstat-base.csv.log", "pidstat-robot.csv.log"]:
                logfiles.append(root + "/" + file)

    fig = make_subplots(
        rows=1,
        cols=2,
        subplot_titles=("Base", "Robot"),
        specs=[[{"secondary_y": True}, {"secondary_y": True}]],
    )
    fig.update_xaxes(title_text="Time [s]")
    fig.update_yaxes(
        title_text="RSS [kB]",
        secondary_y=False,
        titlefont=dict(color="#ab63fa"),
        tickfont=dict(color="#ab63fa"),
    )
    fig.update_yaxes(
        title_text="CPU Usage [%]",
        secondary_y=True,
        titlefont=dict(color="#00cc96"),
        tickfont=dict(color="#00cc96"),
    )
    fig.update_layout(title_text="Resource Usage", showlegend=False)

    for file in logfiles:
        # Get node name
        nodeName = re.search(r"/pidstat-(\w*)", file).group(1)
        column = 1 if ("base" in nodeName) else 2

        # Get number of CPU Cores
        with open(file.replace(".csv", "")) as f:
            cpuCores = int(re.search(r"\((\d*) CPU\)", f.readline()).group(1))

        df = pd.read_csv(file, sep="\s+", usecols=[0, 7, 11, 12])
        # substract start time
        df = df.subtract([simStart, 0, 0, 0], axis="columns")
        # devide CPU usage by core count
        df = df.divide([1, cpuCores, 1, 1], axis="columns")

        # plot
        fig.add_trace(
            go.Scatter(
                x=df["Time"],
                y=df["RSS"],
                mode="lines",
                name="RSS",
                line=dict(color="#ab63fa", width=2),
            ),
            secondary_y=False,
            row=1,
            col=column,
        )
        fig.add_trace(
            go.Scatter(
                x=df["Time"],
                y=df["%CPU"],
                mode="lines",
                name="CPU",
                line=dict(color="#00cc96", width=2),
            ),
            secondary_y=True,
            row=1,
            col=column,
        )

        # Print mean values
        stats = f"Mean Values {nodeName}:\nCPU: \t{df.loc[:,'%CPU'].mean()} %\nRSS: \t{df.loc[:,'RSS'].mean()} kB\n"
        print(stats)

    fig.show()

plotResourceGraphs()


Parsing dtnd stats

In [ ]:
dtndStats = {"base": {}, "robot": {}, "combined": {}}


def loadDtndStats():
    global dtnStats

    # init
    for node in dtndStats:
        dtndStats[node]["sentBundles"] = {}
        dtndStats[node]["receivedBundles"] = {}

    for root, _, files in os.walk(PATH):
        for file in files:
            if file != "dtnd.log":
                continue
            nodeName = re.search(r"\/(\w*)$", root).group(1)
            with open(root + "/" + file) as f:
                content = f.read()

            if nodeName in ["base", "robot"]:

                dtndStats[nodeName]["created"] = len(re.findall(r"Transmission of bundle requested", content))
                dtndStats[nodeName]["transferred"] = len(re.findall(r"Sending bundle succeeded", content))
                dtndStats[nodeName]["relayed"] = len(re.findall(r"Received new bundle", content))
                dtndStats[nodeName]["aborted"] = len(re.findall(r"Sending bundle .+ failed", content))
                dtndStats[nodeName]["dropped"] = len(re.findall(r"Dropping bundle", content))
                dtndStats[nodeName]["removed"] = len(re.findall(r"Removing bundle", content)) - dtndStats[nodeName]["dropped"]
                dtndStats[nodeName]["refused"] = len(re.findall(r"refusing bundle", content))
                dtndStats[nodeName]["delivered"] = len(re.findall(r"Received bundle for local delivery", content))

                tmp = re.findall(r"(\d{4}-\d{2}-\d{2}T.+Z).*Received bundle for local delivery.+\/\/(.+)", content)
                for time, bid in tmp:
                    dtndStats[nodeName]["receivedBundles"][bid] = datetime.fromisoformat(time).timestamp()
                tmp = re.findall(r"(\d{4}-\d{2}-\d{2}T.+Z).*Transmission of bundle requested.+\/\/(.+)", content)
                for time, bid in tmp:
                    dtndStats[nodeName]["sentBundles"][bid] = datetime.fromisoformat(time).timestamp()

            try:
                dtndStats["combined"]["transferred"] = dtndStats["combined"]["transferred"] + len(re.findall(r"Sending bundle succeeded", content))
            except (KeyError):
                dtndStats["combined"]["transferred"] = len(re.findall(r"Sending bundle succeeded", content))
                print("error")

loadDtndStats()


DTN Graps

In [ ]:
def dtnLatency():
    global dtndStats

    sentBundles = {
        **dtndStats["base"]["sentBundles"],
        **dtndStats["robot"]["sentBundles"],
    }
    receivedBundles = {
        **dtndStats["base"]["receivedBundles"],
        **dtndStats["robot"]["receivedBundles"],
    }

    # Calculate latency
    latencyMap = defaultdict(lambda: {"count": 0, "sum": 0})

    # To plot over time, latency values that originate from the same point in time must be averaged
    for bid, time in receivedBundles.items():
        sTime = sentBundles[bid]
        latency = time - sTime
        latencyMap[sTime]["count"] += 1
        latencyMap[sTime]["sum"] += latency
    latencyOverTime = {}
    for time, val in latencyMap.items():
        latencyOverTime[time - simStart] = val["sum"] / val["count"]

    fig = go.Figure()
    fig.update_xaxes(title_text="Time [s]", range=[0, simStop - simStart])
    fig.update_yaxes(
        title_text="E2E Latency [s]",
        titlefont=dict(color="#ab63fa"),
        tickfont=dict(color="#ab63fa"),
    )
    fig.update_layout(title_text="Bidirectional Latency")
    fig.add_trace(
        go.Scatter(
            x=[*latencyOverTime.keys()],
            y=[*latencyOverTime.values()],
            # mode="lines",
            name="RSS",
            line=dict(color="#ab63fa", width=2),
        ),
    )
    fig.show()

    # calc mean latency
    sum = 0
    count = 0
    for val in latencyMap.values():
        sum += val["sum"]
        count += val["count"]

    print(f"Mean Latency: {sum/count}s")


def dtnBundleCount():
    print(f"\nTransfered bundles over all nodes: {dtndStats['combined']['transferred']}")


dtnLatency()
dtnBundleCount()


ROS2DTN Proxy

In [ ]:
def dtnProxyStats():
    topics = []
    sizeDtn = []
    sizeRos = []
    cntDtn = []
    cntRos = []

    for root, _, files in os.walk(PATH):
        if not any(name in root for name in ["base", "robot"]):
            continue

        for file in files:
            if not file.endswith(".json.log"):
                continue
            with open(root + "/" + file) as f:
                jsonObject = json.load(f)

            # topics
            subs = jsonObject["topics"]["subscriber"]
            for topic in jsonObject["topics"]["subscriber"]:
                topics.append(topic)
                cntRos.append(subs[topic]["rosEntry"]["count"])
                cntDtn.append(subs[topic]["dtnEntry"]["count"])
                sizeRos.append(subs[topic]["rosEntry"]["size"])
                sizeDtn.append(subs[topic]["dtnEntry"]["size"])

    fig = make_subplots(
        rows=1, cols=2, subplot_titles=("Message Count", "Message Size")
    )
    fig.update_layout(barmode="group", title_text="Bidirectional Message Statistics")
    fig.update_yaxes(title_text="Message Size [B]", row=1, col=2)
    fig.update_yaxes(title_text="Message Count", row=1, col=1)

    fig.add_trace(
        go.Bar(
            name="ROS",
            x=topics,
            y=cntRos,
            texttemplate="%{y:.r}",
            textposition="outside",
            marker=dict(color="#ab63fa"),
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Bar(
            name="DTN",
            x=topics,
            y=cntDtn,
            texttemplate="%{y:.r}",
            textposition="outside",
            marker=dict(color="#00cc96"),
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Bar(
            name="ROS",
            x=topics,
            y=sizeRos,
            texttemplate="%{y:.r}",
            textposition="outside",
            marker=dict(color="#ab63fa"),
            showlegend=False,
        ),
        row=1,
        col=2,
    )
    fig.add_trace(
        go.Bar(
            name="DTN",
            x=topics,
            y=sizeDtn,
            texttemplate="%{y:.r}",
            textposition="outside",
            marker=dict(color="#00cc96"),
            showlegend=False,
        ),
        row=1,
        col=2,
    )
    fig.show()

    # print stats per topic
    # bandwidth
    print("Transferred Bandwith (DTN payload ONLY)")
    for topic, cnt, size in zip(topics, cntDtn, sizeDtn):
        print(f"{topic}: \t{(cnt * size) / 1000} kB")

    # count
    print("\nMessage Count reduction (ROS -> DTN)")
    for topic, ros, dtn in zip(topics, cntRos, cntDtn):
        print(f"{topic}: \t{ros - dtn} = {((ros-dtn)/ros*100):.2f}%")

    # size
    print("\nMessage Size reduction (ROS -> DTN)")
    for topic, ros, dtn in zip(topics, sizeRos, sizeDtn):
        print(f"{topic}: \t{ros - dtn} Bytes = {((ros-dtn)/ros*100):.2f}%")

dtnProxyStats()
